In [14]:
# Utils packages
import ast
from pathlib import Path, PurePath
from typing import Union

# Data collector packages
from hentai import Utils, Hentai, Option

# Analysis packages
import pandas as pd
from PIL import Image
from tqdm import tqdm

# Compute packages
import numpy as np
import tensorflow as tf

## Data collection

We will use the information and media collected on the nhentai.net website. Recording each entry in a 'metadata.csv' file that aggregates information about the hentai, including links to download pages.

In [15]:
N_SAMPLE = 0 # Number of sample to refetch from hentai.
DATA_PATH = "data" # Directory to drop collected data.

In [16]:
def random_hentai(_: int) -> pd.Series:
    return pd.Series(Utils.get_random_hentai().dictionary(Option.all()))

In [17]:
# Query the nhentai.net API to get N number of random hentai
hentais_df = pd.Series(range(N_SAMPLE))
hentais_df = hentais_df.apply(random_hentai)

# The metadata CSV file that will contain the raw information about the hentai
metadata_path = Path(DATA_PATH) / "raw" / "metadata.csv"

# Create a CSV file with a header only if the file does not exist
if not metadata_path.is_file():
    hentais_df.to_csv(metadata_path, index=False, header="column_names")
else:
    hentais_df.to_csv(metadata_path, index=False, mode="a", header=False)

print("Number of collected hentais: ", len(hentais_df))

<ipython-input-17-4a52f118b4da>:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  hentais_df = pd.Series(range(N_SAMPLE))
Number of collected hentais:  0


## Read dataset file

In [18]:
hentais_df = pd.read_csv(
    Path(DATA_PATH) / "raw" / "metadata.csv",
    converters={
        column_name: ast.literal_eval
        for column_name in [
            "tag",
            "group",
            "parody",
            "character",
            "artist",
            "category",
            "image_urls"
        ]
    }
)
hentais_df

,id,title,scanlator,url,api,media_id,epos,num_favorites,tag,group,parody,character,language,artist,category,cover,thumbnail,image_urls,num_pages
0,212548,Syoku,NaN,https://nhentai.net/g/212548,https://nhentai.net/api/gallery/212548,1133892,1509103975,4452,"[big breasts, milf, nakadashi, sole female, ch...",[hyoco road],[occultic nine],[ririka nishizono],"['english', 'translated']",[hyocorou],[doujinshi],https://t.nhentai.net/galleries/1133892/cover.jpg,https://t.nhentai.net/galleries/1133892/thumb.jpg,[https://i.nhentai.net/galleries/1133892/1.jpg...,33
1,57170,Drug Fumi-chan Seishun Hen Ura Manga Michi,NaN,https://nhentai.net/g/57170,https://nhentai.net/api/gallery/57170,333208,1403992737,21,"[bondage, yuri, schoolgirl uniform, tankoubon]",[],[],[],['japanese'],[shin tsuguru],[manga],https://t.nhentai.net/galleries/333208/cover.jpg,https://t.nhentai.net/galleries/333208/thumb.jpg,"[https://i.nhentai.net/galleries/333208/1.jpg,...",166
2,75495,SHIRONA HAZARD,NaN,https://nhentai.net/g/75495,https://nhentai.net/api/gallery/75495,490119,1403996013,191,[big breasts],[draw go],[pokemon],[cynthia],['japanese'],[souichi],[doujinshi],https://t.nhentai.net/galleries/490119/cover.jpg,https://t.nhentai.net/galleries/490119/thumb.jpg,"[https://i.nhentai.net/galleries/490119/1.jpg,...",23
3,229284,2205-nen no Afterglow,NaN,https://nhentai.net/g/229284,https://nhentai.net/api/gallery/229284,1206999,1522895460,101,"[eyepatch, sole male, sole female]",[frozen world],[touken ranbu],[shokudaikiri mitsutada],['japanese'],[kanaso],[doujinshi],https://t.nhentai.net/galleries/1206999/cover.jpg,https://t.nhentai.net/galleries/1206999/thumb.jpg,[https://i.nhentai.net/galleries/1206999/1.jpg...,47
4,99262,moon marguerite,NaN,https://nhentai.net/g/99262,https://nhentai.net/api/gallery/99262,635686,1404000355,138,"[stockings, paizuri, lingerie, sole female, so...",[meltdown comet],[touhou project],[patchouli knowledge],['japanese'],[yukiu con],[doujinshi],https://t.nhentai.net/galleries/635686/cover.jpg,https://t.nhentai.net/galleries/635686/thumb.jpg,"[https://i.nhentai.net/galleries/635686/1.jpg,...",26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,349482,No. 2 'Rika-shitsu e Youkoso',NaN,https://nhentai.net/g/349482,https://nhentai.net/api/gallery/349482,1856769,1614519177,56,[scanmark],[],[boku wa tomodachi ga sukunai],"[kodaka hasegawa, rika shiguma]",['japanese'],[],[doujinshi],https://t.nhentai.net/galleries/1856769/cover.jpg,https://t.nhentai.net/galleries/1856769/thumb.jpg,[https://i.nhentai.net/galleries/1856769/1.jpg...,31
592,250915,淫獣大小クラン陵辱,NaN,https://nhentai.net/g/250915,https://nhentai.net/api/gallery/250915,1305191,1540380726,111,"[bdsm, collar, lolicon, group, already uploaded]",[aodiso kankou],[macross frontier],[klan klan],['japanese'],[hida mari],[doujinshi],https://t.nhentai.net/galleries/1305191/cover.jpg,https://t.nhentai.net/galleries/1305191/thumb.jpg,[https://i.nhentai.net/galleries/1305191/1.jpg...,18
593,287880,Imouto no Kareshi ga Kawaii no de,NaN,https://nhentai.net/g/287880,https://nhentai.net/api/gallery/287880,1500010,1570893184,1235,"[cheating, shotacon, lolicon, twintails, mosai...",[kuromahou kenkyuujo],[original],[],['japanese'],[nukaji],[doujinshi],https://t.nhentai.net/galleries/1500010/cover.jpg,https://t.nhentai.net/galleries/1500010/thumb.jpg,[https://i.nhentai.net/galleries/1500010/1.jpg...,52
594,99506,Goshujin-sama Anone,NaN,https://nhentai.net/g/99506,https://nhentai.net/api/gallery/99506,636122,1404000399,98,"[dog boy, yaoi, sample, shotacon]",[unagigoya],[],[],['japanese'],[sakura puchilo],[doujinshi],https://t.nhentai.net/galleries/636122/cover.png,https://t.nhentai.net/galleries/636122/thumb.png,"[https://i.nhentai.net/galleries/636122/1.png,...",11


## Remove duplicates

In [19]:
hentais_df = hentais_df.drop_duplicates("id")
hentais_df.to_csv(metadata_path, index=False, header="column_names")

## Download images

In [20]:
def download_hentai(hentai: pd.Series) -> Hentai:
    # A hentai will be stored based on the gallery ID instead of the medai ID
    hentai_path = Path(DATA_PATH) / "raw" / str(hentai.id)

    # Since downloading a hentai is an expensive operation, we only
    # download when the hentai ID directory does not exist
    if not hentai_path.is_dir():
        hentai = Hentai(hentai.id)
        hentai.download(hentai_path, progressbar=True)
        return True

    return False

downloaded_hentais_df = hentais_df.apply(download_hentai, axis=1)

In [21]:
print("Number of downloaded hentais :", downloaded_hentais_df.sum())

Number of downloaded hentais : 0


## Remove corrupted images

In [22]:
def hentai_filenames(hentai: pd.Series):
    return [
        str(PurePath(str(hentai["id"])) / Path(url).name)
        for url in hentai["image_urls"]
    ]

hentais_df = hentais_df.assign(filenames=hentais_df.apply(hentai_filenames, axis=1))
hentais_df

,id,title,scanlator,url,api,media_id,epos,num_favorites,tag,group,parody,character,language,artist,category,cover,thumbnail,image_urls,num_pages,filenames
0,212548,Syoku,NaN,https://nhentai.net/g/212548,https://nhentai.net/api/gallery/212548,1133892,1509103975,4452,"[big breasts, milf, nakadashi, sole female, ch...",[hyoco road],[occultic nine],[ririka nishizono],"['english', 'translated']",[hyocorou],[doujinshi],https://t.nhentai.net/galleries/1133892/cover.jpg,https://t.nhentai.net/galleries/1133892/thumb.jpg,[https://i.nhentai.net/galleries/1133892/1.jpg...,33,"[212548\1.jpg, 212548\2.jpg, 212548\3.jpg, 212..."
1,57170,Drug Fumi-chan Seishun Hen Ura Manga Michi,NaN,https://nhentai.net/g/57170,https://nhentai.net/api/gallery/57170,333208,1403992737,21,"[bondage, yuri, schoolgirl uniform, tankoubon]",[],[],[],['japanese'],[shin tsuguru],[manga],https://t.nhentai.net/galleries/333208/cover.jpg,https://t.nhentai.net/galleries/333208/thumb.jpg,"[https://i.nhentai.net/galleries/333208/1.jpg,...",166,"[57170\1.jpg, 57170\2.jpg, 57170\3.jpg, 57170\..."
2,75495,SHIRONA HAZARD,NaN,https://nhentai.net/g/75495,https://nhentai.net/api/gallery/75495,490119,1403996013,191,[big breasts],[draw go],[pokemon],[cynthia],['japanese'],[souichi],[doujinshi],https://t.nhentai.net/galleries/490119/cover.jpg,https://t.nhentai.net/galleries/490119/thumb.jpg,"[https://i.nhentai.net/galleries/490119/1.jpg,...",23,"[75495\1.jpg, 75495\2.jpg, 75495\3.jpg, 75495\..."
3,229284,2205-nen no Afterglow,NaN,https://nhentai.net/g/229284,https://nhentai.net/api/gallery/229284,1206999,1522895460,101,"[eyepatch, sole male, sole female]",[frozen world],[touken ranbu],[shokudaikiri mitsutada],['japanese'],[kanaso],[doujinshi],https://t.nhentai.net/galleries/1206999/cover.jpg,https://t.nhentai.net/galleries/1206999/thumb.jpg,[https://i.nhentai.net/galleries/1206999/1.jpg...,47,"[229284\1.jpg, 229284\2.png, 229284\3.jpg, 229..."
4,99262,moon marguerite,NaN,https://nhentai.net/g/99262,https://nhentai.net/api/gallery/99262,635686,1404000355,138,"[stockings, paizuri, lingerie, sole female, so...",[meltdown comet],[touhou project],[patchouli knowledge],['japanese'],[yukiu con],[doujinshi],https://t.nhentai.net/galleries/635686/cover.jpg,https://t.nhentai.net/galleries/635686/thumb.jpg,"[https://i.nhentai.net/galleries/635686/1.jpg,...",26,"[99262\1.jpg, 99262\2.jpg, 99262\3.jpg, 99262\..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,349482,No. 2 'Rika-shitsu e Youkoso',NaN,https://nhentai.net/g/349482,https://nhentai.net/api/gallery/349482,1856769,1614519177,56,[scanmark],[],[boku wa tomodachi ga sukunai],"[kodaka hasegawa, rika shiguma]",['japanese'],[],[doujinshi],https://t.nhentai.net/galleries/1856769/cover.jpg,https://t.nhentai.net/galleries/1856769/thumb.jpg,[https://i.nhentai.net/galleries/1856769/1.jpg...,31,"[349482\1.jpg, 349482\2.jpg, 349482\3.jpg, 349..."
592,250915,淫獣大小クラン陵辱,NaN,https://nhentai.net/g/250915,https://nhentai.net/api/gallery/250915,1305191,1540380726,111,"[bdsm, collar, lolicon, group, already uploaded]",[aodiso kankou],[macross frontier],[klan klan],['japanese'],[hida mari],[doujinshi],https://t.nhentai.net/galleries/1305191/cover.jpg,https://t.nhentai.net/galleries/1305191/thumb.jpg,[https://i.nhentai.net/galleries/1305191/1.jpg...,18,"[250915\1.jpg, 250915\2.jpg, 250915\3.jpg, 250..."
593,287880,Imouto no Kareshi ga Kawaii no de,NaN,https://nhentai.net/g/287880,https://nhentai.net/api/gallery/287880,1500010,1570893184,1235,"[cheating, shotacon, lolicon, twintails, mosai...",[kuromahou kenkyuujo],[original],[],['japanese'],[nukaji],[doujinshi],https://t.nhentai.net/galleries/1500010/cover.jpg,https://t.nhentai.net/galleries/1500010/thumb.jpg,[https://i.nhentai.net/galleries/1500010/1.jpg...,52,"[287880\1.jpg, 287880\2.jpg, 287880\3.jpg, 287..."
594,99506,Goshujin-sama Anone,NaN,https://nhentai.net/g/99506,https://nhentai.net/api/gallery/99506,636122,1404000399,98,"[dog boy, yaoi, sample, shotacon]",[u

In [23]:
def preprocess_image(filename: str) -> Union[os.PathLike, None]:
    image_path = Path(filename)
    image_preproc_dir = Path(DATA_PATH) / "preprocessed" / image_path.parent
    image_preproc_path = image_preproc_dir / Path(image_path.stem + ".jpg")

    # Preprocess only those images that do not exist in
    # the destination dataset for performance reason
    if not image_preproc_path.is_file():
        try:
            image = Image.open(Path(DATA_PATH) / "raw" / image_path)
            # Normalize images to be JPEG compatible
            image = image.convert("RGB")
            image_preproc_dir.mkdir(parents=True, exist_ok=True)
            image.save(image_preproc_path, "JPEG")
        except:
            return None

    return str(PurePath(image_preproc_path))

def preprocess_images(hentai: pd.Series) -> bool:
    return [
        preprocess_image(filename)
        for filename in hentai["filenames"]
    ]

tqdm.pandas()
hentais_df = hentais_df.assign(filenames=hentais_df.progress_apply(preprocess_images, axis=1))

C:\Anaconda3\lib\site-packages\tqdm\std.py:702: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 596/596 [00:03<00:00, 177.39it/s]


We will filter every hentais that contain at least one corrupted images, since pages have order dependencies.

In [24]:
hentais_df = hentais_df[hentais_df["filenames"].apply(all)]

In [25]:
# The metadata CSV file that will contain the raw information about the hentai
metadata_path = Path(DATA_PATH) / "preprocessed" / "metadata.csv"
hentais_df.to_csv(
    metadata_path,
    index=False,
    header="column_names",
    columns=[
        "id",
        "title",
        "num_favorites",
        "tag",
        "group",
        "parody",
        "character",
        "language",
        "artist",
        "category",
        "num_pages",
        "filenames"
    ]
)
print("Number of preprocessed images :", len(hentais_df))

Number of preprocessed images : 595
